In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import re
import random
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)


In [4]:
user_list = pd.read_csv('./data/naver_user.csv')
user_list.head(5)

,reviewNo,userId
0,15772038,airf
1,15772037,nanw
2,15772036,zxcv
3,15772035,sdh1
4,15772032,guan


In [13]:
df_rating = pd.DataFrame(columns=("userId", "reviewNo", "rating", "movieId"))

In [14]:
def get_user_list(limit):
    page = 1
    user_list = []
    while len(user_list) <= limit:
        driver.get('https://movie.naver.com/movie/point/af/list.nhn?&page={0}'.format(page))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        reviewNum = soup.select('#old_content > table > tbody > tr > td.ac.num')
        userId = soup.select('table.list_netizen > tbody > tr > td > a.author')
        print(reviewNum)
        for i in range(len(reviewNum)):
            review = str(reviewNum[i]).replace('<td class="ac num">','').replace('</td>','')
            user = userId[i].text.replace('*','')
            if user not in [user[1] for user in user_list]:
                user_list.append((review, user))

        time.sleep(random.randrange(2, 5))
        page += 1

    return user_list[:limit]

In [15]:
def get_movie_link(soup):
    movie_links = soup.select('a[href]')

    movie_links_list = []
    for link in movie_links:
        if re.search(r'st=mcode&sword' and r'&target=after$', link['href']):
            target_url = 'https://movie.naver.com/movie/point/af/list.nhn'+str(link['href'])
            movie_links_list.append(target_url)
    
    return movie_links_list[1:]

In [16]:
def get_review_list(length):
    for i in range(length):
        page = 1
        userId = user_list['userId'][i]
        driver.get('https://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword='+str(user_list['reviewNo'][i])+'&target=after&page={0}'.format(page))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        rating_list = soup.select('#old_content > table > tbody > tr > td.point')
        reviewNum = soup.select('#old_content > table > tbody > tr > td.ac.num')
        movieId_list = get_movie_link(soup)
        total = soup.select('#old_content > h5.sub_tlt > div.h5_right_txt > strong')
        
        total = str(total[0]).replace('<strong class="c_88 fs_11">', '').replace('</strong>', '')
        count = int(int(total)-10)
        
        for j in range(len(rating_list)):
            rating = str(rating_list[j]).replace('<td class="point">', '').replace('</td>', '')
            review = str(reviewNum[j]).replace('<td class="ac num">','').replace('</td>','')
            movieId = str(movieId_list[j]).replace('https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=','').replace('&target=after','')
            df_rating.loc[len(df_rating)] = [userId, review, rating, movieId]
        
        # page 가 더 있다면
        while count > 0:
            page = page+1
            count = count - 10
            driver.get('https://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword='+str(user_list['reviewNo'][i])+'&target=after&page={0}'.format(page))
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
        
            rating_list = soup.select('#old_content > table > tbody > tr > td.point')
            reviewNum = soup.select('#old_content > table > tbody > tr > td.ac.num')
            
            movieId_list = get_movie_link(soup)
            total = soup.select('#old_content > h5.sub_tlt > div.h5_right_txt > strong')
            
            for j in range(len(rating_list)):
                rating = str(rating_list[j]).replace('<td class="point">', '').replace('</td>', '')
                review = str(reviewNum[j]).replace('<td class="ac num">','').replace('</td>','')
                movieId = str(movieId_list[j]).replace('https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=','').replace('&target=after','')
                df_rating.loc[len(df_rating)] = [userId, review, rating, movieId]
        
            time.sleep(random.randrange(2, 5))    
            
        time.sleep(random.randrange(2, 5))    

    return None

In [17]:
get_review_list(len(user_list))
df_rating.head(10)
#print(get_user_list(5))
#driver.close()




,userId,reviewNo,rating,movieId
0,airf,15772038,2,136900
1,airf,15733971,10,163788
2,airf,15650791,10,174065
3,nanw,15772037,10,154667
4,nanw,15623438,10,136900
5,nanw,11909074,10,145335
6,nanw,10303746,9,120141
7,nanw,3809124,9,54704
8,zxcv,15778201,1,174065
9,zxcv,15772036,7,113351


In [18]:
len(df_rating)

5139

In [19]:
df_rating.to_csv('./data/rating.csv', index=False)

In [42]:
test = pd.read_csv('./data/rating.csv')
print(len(test))
test.head(5)

5139


,userId,reviewNo,rating,movieId
0,airf,15772038,2,136900
1,airf,15733971,10,163788
2,airf,15650791,10,174065
3,nanw,15772037,10,154667
4,nanw,15623438,10,136900


In [43]:
test.loc[len(test)] = ['glab',15772028,8,137327]
test.loc[len(test)] = ['patl',15771989,10,161967]
test.loc[len(test)] = ['yoya',15771960,10,161967]
test.loc[len(test)] = ['ebc8',15771979,1,174065]
test.loc[len(test)] = ['imag',15771940,10,161967]
print(len(test))

5144


In [44]:
test = test.loc[test.reviewNo <= 15772038].sort_values(["userId"]).reset_index(drop=True)

In [45]:
print(len(test))
test.head(10)

5124


,userId,reviewNo,rating,movieId
0,0424,15772026,10,161967
1,2pop,15771991,10,177483
2,airf,15772038,2,136900
3,airf,15733971,10,163788
4,airf,15650791,10,174065
5,akh3,14709561,10,163533
6,akh3,14709563,10,167697
7,akh3,14709564,8,164115
8,akh3,14709565,10,171414
9,akh3,14709566,10,136315


In [46]:
test = test.drop_duplicates(subset=['userId', 'movieId'], keep='first').reset_index(drop=True)
len(test)
test.head(-1)

,userId,reviewNo,rating,movieId
0,0424,15772026,10,161967
1,2pop,15771991,10,177483
2,airf,15772038,2,136900
3,airf,15733971,10,163788
4,airf,15650791,10,174065
5,akh3,14709561,10,163533
6,akh3,14709563,10,167697
7,akh3,14709564,8,164115
8,akh3,14709565,10,171414
9,akh3,14709566,10,136315


In [47]:
test.to_csv('./data/final_rating.csv', index=False)